In [1]:
import tensorflow as tf
from simulator import Sim
import numpy as np
import rospy

In [2]:
rospy.init_node('trainer', anonymous=True)
sim = Sim()
data = sim.states
label = sim.actions

def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation,:,:]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels

s_data, s_label = randomize(data,label)


train_data = s_data[:,:].reshape(-1,12).astype(np.float32)
train_label = s_label[:800,:].reshape(-1,3).astype(np.float32)
test_data = s_data[800:,:].reshape(-1,12).astype(np.float32)
test_label = s_label[800:,:].reshape(-1,3).astype(np.float32)

train_data = train_data[:,9:]
test_data = test_data[:,9:]

In [11]:
graph = tf.Graph()
alpha = 0.02
hid_num1 = 500
hid_num2 = 100

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(None, 3))
    tf_train_labels = tf.placeholder(tf.float32, shape=(None, 3))
    tf_test_dataset = tf.constant(test_data)
    tf_test_labels = tf.constant(test_label)
    
    F1_weights = tf.Variable(tf.truncated_normal([3,hid_num1], stddev=10.0))
    F1_biases = tf.Variable(tf.constant(1.0, shape=[hid_num1]))
    
    F2_weights = tf.Variable(tf.truncated_normal([hid_num1, hid_num2], stddev=5.0))
    F2_biases = tf.Variable(tf.constant(1.0, shape=[hid_num2]))
    
    F3_weights = tf.Variable(tf.truncated_normal([hid_num2,3], stddev=1.0))
    F3_biases = tf.Variable(tf.constant(1.0, shape=[1]))
    
    def model(data):
        fc = tf.matmul(data, F1_weights)
        hidden = tf.nn.relu(fc + F1_biases)
        
        fc = tf.matmul(hidden, F2_weights)
        hidden = tf.nn.sigmoid(fc + F2_biases)
        
        fc = tf.matmul(hidden, F3_weights)
        output = fc + F3_biases
    
        return output
    
    train_pred = model(tf_train_dataset)
    loss = tf.losses.mean_squared_error(labels=tf_train_labels, predictions=train_pred) \
        + alpha*tf.nn.l2_loss(F1_weights)+alpha*tf.nn.l2_loss(F2_weights)
    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
    #optimizer = tf.train.RMSPropOptimizer(0.001).minimize(loss)
    #optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(loss)
    
    
    test_pred = model(tf_test_dataset)
    test_loss = tf.losses.mean_squared_error(labels=tf_test_labels, predictions=test_pred)

In [12]:
num_steps = 100000
batch_size = 10
config = tf.ConfigProto()
config.log_device_placement = True
with tf.Session(graph=graph, config = config) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_label.shape[0] - batch_size)
        batch_data = train_data[offset:(offset + batch_size), :]
        batch_labels = train_label[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l = session.run(
          [optimizer, loss], feed_dict=feed_dict)
        if (step % 2000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('--------------------------------------')
    print('Test loss: %.1f%%' % test_loss.eval())
    test_rslt = test_pred.eval()
    i_test = 0
    while(i_test!=''):
        try:
            i_test = input("Input an index of test image (or Enter to quit): ")
            label = test_label[int(i_test),:].tolist()
            rslt = test_rslt[i_test,:]
            print label
            print rslt
        except:
            break

Initialized
Minibatch loss at step 0: 11033.369141
--------------------------------------
Minibatch loss at step 2000: 5588.299316
--------------------------------------
Minibatch loss at step 4000: 2777.424805
--------------------------------------
Minibatch loss at step 6000: 1340.372803
--------------------------------------
Minibatch loss at step 8000: 677.764648
--------------------------------------
Minibatch loss at step 10000: 411.852661
--------------------------------------
Minibatch loss at step 12000: 300.866638
--------------------------------------
Minibatch loss at step 14000: 243.382446
--------------------------------------
Minibatch loss at step 16000: 207.075348
--------------------------------------
Minibatch loss at step 18000: 183.495178
--------------------------------------
Minibatch loss at step 20000: 169.198181
--------------------------------------
Minibatch loss at step 22000: 156.933304
--------------------------------------
Minibatch loss at step 24000: 1